# Keras Example

Using an SMS Spam data set (slightly modified) from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data set is a collection of 5574 SMS messages that have been labeled as ham or spam. The file is a tab-delimited file with the first column the label and the second the message content. I edited the data set to remove some unwanted columns and add headings. 



In [1]:
# some necessary packages
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
import pickle
import numpy as np
import pandas as pd

# set seed for reproducibility
np.random.seed(1234)

Using TensorFlow backend.


In [2]:
df = pd.read_csv('../data/sms-spam.csv', header=0, usecols=[1,2], encoding='latin-1')
print('rows and columns:', df.shape)
print(df.head())

rows and columns: (4837, 2)
   spam                                               text
0     0  Go until jurong point, crazy.. Available only ...
1     0                      Ok lar... Joking wif u oni...
2     1  Free entry in 2 a wkly comp to win FA Cup fina...
3     0  U dun say so early hor... U c already then say...
4     0  Nah I don't think he goes to usf, he lives aro...


In [3]:
# split df into train and test
i = np.random.rand(len(df)) < 0.8
train = df[i]
test = df[~i]
print("train data size: ", train.shape)
print("test data size: ", test.shape)

train data size:  (3856, 2)
test data size:  (981, 2)


In [4]:
# set up X and Y
num_labels = 2
vocab_size = 25000
batch_size = 100

# fit the tokenizer on the training data
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train.text)

x_train = tokenizer.texts_to_matrix(train.text, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test.text, mode='tfidf')

encoder = LabelEncoder()
encoder.fit(train.spam)
y_train = encoder.transform(train.spam)
y_test = encoder.transform(test.spam)

# check shape
print("train shapes:", x_train.shape, y_train.shape)
print("test shapes:", x_test.shape, y_test.shape)
print("test first five labels:", y_test[:5])


train shapes: (3856, 25000) (3856,)
test shapes: (981, 25000) (981,)
test first five labels: [0 1 1 1 0]


In [5]:
# fit Keras model
model = Sequential()
model.add(Dense(32, input_dim=vocab_size, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
 
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)

W1114 17:15:13.621602 139652879566656 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1114 17:15:13.888079 139652879566656 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Train on 3470 samples, validate on 386 samples
Epoch 1/30
3470/3470 [==============================] - 1s 263us/step - loss: 0.5447 - accuracy: 0.8418 - val_loss: 0.4008 - val_accuracy: 0.8938
Epoch 2/30
3470/3470 [==============================] - 1s 162us/step - loss: 0.2937 - accuracy: 0.9248 - val_loss: 0.2233 - val_accuracy: 0.9534
Epoch 3/30
3470/3470 [==============================] - 1s 164us/step - loss: 0.1367 - accuracy: 0.9833 - val_loss: 0.1269 - val_accuracy: 0.9793
Epoch 4/30
3470/3470 [==============================] - 1s 157us/step - loss: 0.0640 - accuracy: 0.9968 - val_loss: 0.0950 - val_accuracy: 0.9845
Epoch 5/30
3470/3470 [==============================] - 1s 157us/step - loss: 0.0363 - accuracy: 0.9974 - val_loss: 0.0847 - val_accuracy: 0.9870
Epoch 6/30
3470/3470 [==============================] - 1s 161us/step - loss: 0.0228 - accuracy: 0.9986 - val_loss: 0.0810 - val_accuracy: 0.9870
Epoch 7/30
3470/3470 [==============================] - 1s 161us/step - loss:

In [6]:
# evaluate
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('Accuracy: ', score[1])

981/981 [==============================] - 0s 124us/step
Accuracy:  0.9847095012664795


In [7]:
print(score)

[0.14624296876466117, 0.9847095012664795]


In [9]:
# get predictions so we can calculate more metrics
pred = model.predict_classes(x_test)


In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('accuracy score: ', accuracy_score(y_test, pred))
print('precision score: ', precision_score(y_test, pred))
print('recall score: ', recall_score(y_test, pred))
print('f1 score: ', f1_score(y_test, pred))

accuracy score:  0.9847094801223242
precision score:  0.9916666666666667
recall score:  0.8947368421052632
f1 score:  0.9407114624505929


This was by far the most accurate. Much more work could be done modifying the network topology. 